In [1]:
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn.metrics import mean_absolute_percentage_error
from deap import base, creator, tools, algorithms

In [2]:
SEED = 42
random.seed(SEED)
np.random.seed(SEED)   

data = pd.read_csv('DATA_KOTA.csv')

df = pd.DataFrame(data)

# Membagi kolom tanggal menjadi tahun dan bulan
df['tahun'] = df['tahun_bulan'].apply(lambda x: int(x.split('-')[0]))
df['bulan'] = df['tahun_bulan'].apply(lambda x: int(x.split('-')[1]))

# Normalisasi MinMax
scaler = MinMaxScaler()
df['jumlah_kecelakaan'] = scaler.fit_transform(df[['jumlah_kecelakaan']])

# Memisahkan fitur dan target
X = df[['tahun', 'bulan']].values
y = df['jumlah_kecelakaan'].values

In [3]:
# Pisahkan data menjadi training dan testing
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=SEED)

In [4]:
# Fungsi untuk menghitung MAPE menggunakan model SVR dengan parameter tertentu
def calculate_mape(X_train, y_train, X_val, y_val, params):
    C, epsilon, gamma = params
    svr = SVR(kernel='linear', C=C, epsilon=epsilon, gamma=gamma)
    svr.fit(X_train, y_train)
    y_pred = svr.predict(X_val)
    mape = mean_absolute_percentage_error(y_val, y_pred) *100
    return mape

# Inisialisasi populasi awal
def initialize_population(population_size, parameter_ranges):
    population = []
    for _ in range(population_size):
        chromosome = []
        for param_range in parameter_ranges:
            param_value = np.random.uniform(param_range[0], param_range[1])
            chromosome.append(param_value)
        population.append(chromosome)
    return population

# Seleksi individu berdasarkan nilai fitness (MAPE)
def selection(population, X_train, y_train, X_val, y_val):
    selected_population = []
    for chromosome in population:
        mape = calculate_mape(X_train, y_train, X_val, y_val, chromosome)
        selected_population.append((chromosome, mape))
    selected_population.sort(key=lambda x: x[1])  # Sort berdasarkan MAPE terendah
    return selected_population[:len(population)//2]  # Pilih setengah individu terbaik

# Crossover: Menghasilkan keturunan dari pasangan individu terpilih
def crossover(selected_population):
    new_population = []
    ukuran_populasi = len(selected_population)
    
    for i in range(ukuran_populasi):
        parent1 = selected_population[i][0]
        
        # Tentukan parent2 dengan pemeriksaan batas
        if i % 2 == 0:
            if i - 1 >= 0:
                parent2 = selected_population[i-1][0]
            else:
                parent2 = selected_population[i][0]  # Penanganan alternatif jika i-1 tidak valid
        else:
            if i + 1 < ukuran_populasi:
                parent2 = selected_population[i+1][0]
            else:
                parent2 = selected_population[i][0]  # Penanganan alternatif jika i+1 tidak valid
        
        crossover_point = np.random.randint(1, len(parent1)-1)
        child = parent1[:crossover_point] + parent2[crossover_point:]
        new_population.append(child)
    
    return new_population

# Mutasi: Mengubah nilai gen secara acak pada populasi baru
def mutation(new_population, mutation_rate, parameter_ranges, sigma_range):
    for i in range(len(new_population)):
        if np.random.random() < mutation_rate:
            mutated_gene_index = np.random.randint(0, len(new_population[i]))
            sigma = np.random.uniform(sigma_range[0], sigma_range[1])
            new_value = new_population[i][mutated_gene_index] + np.random.normal(0, sigma)
            new_population[i][mutated_gene_index] = np.clip(new_value, parameter_ranges[mutated_gene_index][0], parameter_ranges[mutated_gene_index][1])
    return new_population

In [5]:
# Rentang nilai parameter optimal
parameter_ranges = [(1, 5), (0.001, 0.1), (1e-07, 0.0001)]

# Inisialisasi populasi awal
population_size = 50
population = initialize_population(population_size, parameter_ranges)

# Iterasi algoritma genetika
crossover_rate = 0.3  # Tetapkan tingkat crossover tetap untuk eksperimen
mutation_rate = 0.7  # Tetapkan tingkat mutasi tetap untuk eksperimen
num_generations = 25  # Tetapkan jumlah generasi tetap untuk eksperimen
sigma_range = (0.5, 5)

for generation in range(num_generations):
    print(f"Generasi {generation + 1}:")
    selected_population = selection(population, X_train, y_train, X_val, y_val)
    for i, (chromosome, mape) in enumerate(selected_population):
        print(f"Individu {i + 1}: C={chromosome[0]}, epsilon={chromosome[1]}, gamma={chromosome[2]}, MAPE={mape}")
    
    new_population = crossover(selected_population)
    new_population = mutation(new_population, mutation_rate, parameter_ranges, sigma_range)

    # Adjust the population size if necessary
    if len(new_population) < population_size:
        additional_individuals = initialize_population(population_size - len(new_population), parameter_ranges)
        new_population.extend(additional_individuals)
    elif len(new_population) > population_size:
        new_population = new_population[:population_size]
    
    # Make sure the new population has the correct size
    assert len(new_population) == population_size, "Ukuran populasi baru tidak sesuai"
    
    population = new_population

# Mengambil individu terbaik (dengan MAPE terendah) dari populasi terakhir
best_chromosome, best_mape = min(selected_population, key=lambda x: x[1])

# Menguji model regresi dengan parameter terbaik
best_svr = SVR(kernel='linear', C=best_chromosome[0], epsilon=best_chromosome[1], gamma=best_chromosome[2])
best_svr.fit(X_train, y_train)
y_pred = best_svr.predict(X_val)
mape = mean_absolute_percentage_error(y_val, y_pred) *100

print("\nParameter terbaik:")
print(f"C: {best_chromosome[0]}")
print(f"Epsilon: {best_chromosome[1]}")
print(f"Gamma: {best_chromosome[2]}")
print(f"MAPE terbaik: {best_mape}")

print("\nHasil peramalan:")
print("Prediksi:", y_pred)
print("MAPE:", mape)

Generasi 1:
Individu 1: C=3.0909313175279762, epsilon=0.04332656081749642, gamma=2.639370761735109e-06, MAPE=24.690530984654266
Individu 2: C=3.042989210310263, epsilon=0.042323689311729126, gamma=2.228857026602595e-05, MAPE=24.782683053401104
Individu 3: C=2.0071291833014566, epsilon=0.05022760208334616, gamma=3.0157743150695288e-05, MAPE=25.057393884433566
Individu 4: C=1.9971689165954998, epsilon=0.04162790938052734, gamma=7.557955874045057e-05, MAPE=25.20179302394437
Individu 5: C=2.2574239243053067, epsilon=0.051348498425305575, gamma=9.07658907452167e-05, MAPE=25.240287154426245
Individu 6: C=2.292811728083021, epsilon=0.05236027155259325, gamma=7.033159399362827e-05, MAPE=25.4368561441425
Individu 7: C=2.454518409517176, epsilon=0.09720642618937511, gamma=9.624848476471691e-05, MAPE=26.408965200108536
Individu 8: C=4.7955421490133325, epsilon=0.09659757127438139, gamma=8.085889507683447e-05, MAPE=26.554448877928323
Individu 9: C=2.49816047538945, epsilon=0.0951207163345817, gamm

In [6]:
pred_denormalized = scaler.inverse_transform(y_pred.reshape(-1, 1))
print(pred_denormalized)

[[ 3.98305588]
 [ 3.99261566]
 [ 9.98496773]
 [10.00217495]
 [ 5.98560517]
 [11.99707642]
 [ 8.00153778]
 [11.98560502]
 [ 5.98369297]
 [12.00281224]
 [10.00408682]
 [11.98942868]]


In [7]:
# Menguji model regresi dengan parameter terbaik
best_svr = SVR(kernel='poly', C=best_chromosome[0], epsilon=best_chromosome[1], gamma=best_chromosome[2])
best_svr.fit(X_train, y_train)
y_pred = best_svr.predict(X_val)
mape = mean_absolute_percentage_error(y_val, y_pred) *100

print("\nHasil peramalan:")
print("Prediksi:", y_pred)
print("MAPE:", mape)


Hasil peramalan:
Prediksi: [-9.49574498 -3.61774758  2.28920253 12.94716036 -4.3996867  13.31447005
  9.00447979  6.22529715 -5.57131634 16.89373698 14.14421591  8.5780796 ]
MAPE: 1871.9034147523348


In [8]:
pred_denormalized = scaler.inverse_transform(y_pred.reshape(-1, 1))
print(pred_denormalized)

[[-208.90638956]
 [ -79.59044679]
 [  50.36245576]
 [ 284.83752785]
 [ -96.79310741]
 [ 292.91834102]
 [ 198.09855537]
 [ 136.95653723]
 [-122.56895944]
 [ 371.6622136 ]
 [ 311.17274992]
 [ 188.7177513 ]]


In [9]:
# Menguji model regresi dengan parameter terbaik
best_svr = SVR(kernel='rbf', C=best_chromosome[0], epsilon=best_chromosome[1], gamma=best_chromosome[2])
best_svr.fit(X_train, y_train)
y_pred = best_svr.predict(X_val)
mape = mean_absolute_percentage_error(y_val, y_pred) *100

print("\nHasil peramalan:")
print("Prediksi:", y_pred)
print("MAPE:", mape)


Hasil peramalan:
Prediksi: [0.332756   0.32487671 0.37218086 0.3580078  0.34434412 0.37600943
 0.34483542 0.38531384 0.34589456 0.37117358 0.35638332 0.382275  ]
MAPE: 40.40920704474487


In [10]:
pred_denormalized = scaler.inverse_transform(y_pred.reshape(-1, 1))
print(pred_denormalized)

[[7.32063208]
 [7.14728752]
 [8.18797897]
 [7.87617158]
 [7.57557054]
 [8.27220746]
 [7.58637915]
 [8.47690453]
 [7.60968024]
 [8.16581873]
 [7.84043298]
 [8.41005005]]


In [11]:
# Menguji model regresi dengan parameter terbaik
best_svr = SVR(kernel='sigmoid', C=best_chromosome[0], epsilon=best_chromosome[1], gamma=best_chromosome[2])
best_svr.fit(X_train, y_train)
y_pred = best_svr.predict(X_val)
mape = mean_absolute_percentage_error(y_val, y_pred) *100

print("\nHasil peramalan:")
print("Prediksi:", y_pred)
print("MAPE:", mape)


Hasil peramalan:
Prediksi: [0.36338592 0.36338592 0.36338592 0.36338592 0.36338592 0.36338592
 0.36338592 0.36338592 0.36338592 0.36338592 0.36338592 0.36338592]
MAPE: 43.44126875125478


In [12]:
pred_denormalized = scaler.inverse_transform(y_pred.reshape(-1, 1))
print(pred_denormalized)

[[7.99449015]
 [7.99449015]
 [7.99449015]
 [7.99449015]
 [7.99449015]
 [7.99449015]
 [7.99449015]
 [7.99449015]
 [7.99449015]
 [7.99449015]
 [7.99449015]
 [7.99449015]]
